In [ ]:
include("_.jl")

In [ ]:
an_hu_me = OnePiece.io.table.read(joinpath(PAO, "human_gene_by_method.tsv"))

hu_ = an_hu_me[!, "Gene"]

In [ ]:
hg = OnePiece.gene.read_hgnc()

hg_gr = OnePiece.extension.dataframe.map_to_column(coalesce.(hg, "?"), ["symbol", "gene_group"])

;

In [ ]:
pa1 = "set1_to_genes.json"

pa2 = "set2_to_genes.json"

na1 = "Human Immune Population"

na2 = "Brain Cell Type"

na1c = OnePiece.extension.path.clean(na1)

na2c = OnePiece.extension.path.clean(na2)

se1_ge_ = OnePiece.extension.dict.read(joinpath(PAI, pa1))

se2_ge_ = OnePiece.extension.dict.read(joinpath(PAI, pa2))

ge1_ = vcat(values(se1_ge_)...)

ge2_ = vcat(values(se2_ge_)...)

println(length(ge1_))

println(length(ge2_))

In [ ]:
na = na1

nac = na1c

se_ge_ = se1_ge_

ge_ = ge1_

length(ge_)

In [ ]:
na = string(na1, " and ", na2)

nac = OnePiece.extension.path.clean(na)

se_ge_ = sort(merge(se1_ge_, se2_ge_))

ge_ = vcat(values(se_ge_)...)

length(ge_)

In [ ]:
co_ = intersect(hu_, ge_)

In [ ]:
se_ = [join([se for (se, ge_) in se_ge_ if co in ge_], " & ") for co in co_]

In [ ]:
gr_ = []

for co in co_

    if haskey(hg_gr, co)

        gr = hg_gr[co]

    else

        println(co)

        gr = "??"

    end

    push!(gr_, gr)

end

gr_

In [ ]:
an_ge_ir = sort(DataFrame("Gene" => co_, na => se_, "Group" => gr_), na)

se_ = contains.(an_ge_ir[!, na], "&")

if any(se_)

    println("Selecting genes found in both sets")

    an_ge_ir = an_ge_ir[se_, :]

end

OnePiece.io.table.write(joinpath(PAO, string(nac, "_gene_by_information.tsv")), an_ge_ir)

an_ge_ir

## Plot

In [ ]:
he = 1600

wi = round(Int, he * MathConstants.golden)

axis_title_font_size = 24

la = Layout(
    barmode = "stack",
    height = he,
    width = wi,
    margin_b = 240,
    title = attr(text = string("Number of Human Genes in ", na, "s"), font_size = 32),
    yaxis_title = attr(text = "Number of Human Genes", font_size = axis_title_font_size),
    xaxis = attr(title = attr(text = na, font_size = axis_title_font_size), tickangle = 60),
)

;

In [ ]:
se_n = sort(countmap(an_ge_ir[!, na]); byvalue = true)

In [ ]:
x = collect(keys(se_n))

y = collect(values(se_n))

pl = OnePiece.figure.plot_bar([y], [x], la = la)

display(pl)

OnePiece.figure.write(joinpath(PAO, string(nac, "_genes.png")), pl, he, wi, 8)

In [ ]:
name_ = []

y_ = []

id = 1

for gr in unique(an_ge_ir[!, "Group"])

    if gr in ["?", "??"]

        #continue

    end

    #gr = split(gr, "|")[id]

    #println(gr)

    push!(name_, gr)

    n_be_ = []

    for se in x

        da = an_ge_ir[an_ge_ir[!, na] .== se, :]

        gr_ = da[!, "Group"]

        #gr_ = getindex.(split.(gr, "|"), id)

        n_be = sum(gr_ .== gr)

        if 0 < n_be

            #println("-"^99)
            #
            #println(da)
            #
            #println(n_be)
            #
            #println()

        end

        push!(n_be_, n_be)

    end

    push!(y_, n_be_)

end

In [ ]:
pl = OnePiece.figure.plot_bar(y_, fill(x, length(y_)); name_ = name_, la = la)

display(pl)

OnePiece.figure.write(joinpath(PAO, string(nac, "_genes_by_group.png")), pl, he, wi, 8)